In [2]:
import math
from typing import Dict, Any, List
import pandas as pd
import yfinance as yf


# =========================================
# 1. Core helpers and scorecard functions
# =========================================
def safe_div(numerator: float, denominator: float) -> float:
    """
    Safely perform division, returning NaN if denominator is 0 or None.
    """
    if denominator is None or denominator == 0:
        return math.nan
    return numerator / denominator


def calculate_ratios(borrower: Dict[str, Any]) -> Dict[str, float]:
    """
    Calculate key credit ratios for a single borrower.
    Expected keys in `borrower`:
        ebitda, ebit, interest_expense,
        total_debt, current_assets, current_liabilities,
        annual_debt_service
    """
    ebitda = borrower.get("ebitda", 0.0)
    ebit = borrower.get("ebit", 0.0)
    interest_expense = borrower.get("interest_expense", 0.0)
    total_debt = borrower.get("total_debt", 0.0)
    current_assets = borrower.get("current_assets", 0.0)
    current_liabilities = borrower.get("current_liabilities", 0.0)
    annual_debt_service = borrower.get("annual_debt_service", 0.0)

    dscr = safe_div(ebitda, annual_debt_service)
    icr = safe_div(ebit, interest_expense)
    debt_ebitda = safe_div(total_debt, ebitda)
    current_ratio = safe_div(current_assets, current_liabilities)

    return {
        "dscr": dscr,
        "interest_coverage": icr,
        "debt_ebitda": debt_ebitda,
        "current_ratio": current_ratio,
    }


def score_dscr(dscr: float) -> int:
    if math.isnan(dscr):
        return 0
    if dscr >= 2.0:
        return 5
    elif dscr >= 1.5:
        return 4
    elif dscr >= 1.2:
        return 3
    elif dscr >= 1.0:
        return 2
    else:
        return 1


def score_interest_coverage(icr: float) -> int:
    if math.isnan(icr):
        return 0
    if icr >= 5.0:
        return 5
    elif icr >= 4.0:
        return 4
    elif icr >= 2.5:
        return 3
    elif icr >= 1.5:
        return 2
    else:
        return 1


def score_debt_ebitda(debt_ebitda: float) -> int:
    if math.isnan(debt_ebitda):
        return 0
    if debt_ebitda <= 1.0:
        return 5
    elif debt_ebitda <= 2.0:
        return 4
    elif debt_ebitda <= 3.5:
        return 3
    elif debt_ebitda <= 5.0:
        return 2
    else:
        return 1


def score_current_ratio(cr: float) -> int:
    if math.isnan(cr):
        return 0
    if cr >= 2.0:
        return 5
    elif cr >= 1.5:
        return 4
    elif cr >= 1.2:
        return 3
    elif cr >= 1.0:
        return 2
    else:
        return 1


def calculate_weighted_score(scores: Dict[str, int]) -> float:
    weights = {
        "dscr_score": 0.35,
        "debt_ebitda_score": 0.30,
        "current_ratio_score": 0.20,
        "interest_coverage_score": 0.15,
    }

    total = 0.0
    weight_sum = 0.0

    for key, w in weights.items():
        s = scores.get(key, 0)
        if s > 0:
            total += s * w
            weight_sum += w

    if weight_sum == 0:
        return math.nan

    return total / weight_sum


def score_to_rating(score: float) -> str:
    if math.isnan(score):
        return "NR"
    if score >= 4.5:
        return "AAA"
    elif score >= 4.0:
        return "AA"
    elif score >= 3.5:
        return "A"
    elif score >= 3.0:
        return "BBB"
    elif score >= 2.5:
        return "BB"
    elif score >= 2.0:
        return "B"
    else:
        return "CCC"


def rating_to_risk_band(rating: str) -> str:
    if rating in ("AAA", "AA", "A"):
        return "Low"
    elif rating in ("BBB", "BB"):
        return "Medium"
    elif rating in ("B", "CCC"):
        return "High"
    else:
        return "Unknown"


def evaluate_borrower(borrower: Dict[str, Any]) -> Dict[str, Any]:
    ratios = calculate_ratios(borrower)

    dscr_score = score_dscr(ratios["dscr"])
    icr_score = score_interest_coverage(ratios["interest_coverage"])
    de_score = score_debt_ebitda(ratios["debt_ebitda"])
    cr_score = score_current_ratio(ratios["current_ratio"])

    scores = {
        "dscr_score": dscr_score,
        "interest_coverage_score": icr_score,
        "debt_ebitda_score": de_score,
        "current_ratio_score": cr_score,
    }

    total_score = calculate_weighted_score(scores)
    rating = score_to_rating(total_score)
    risk_band = rating_to_risk_band(rating)

    result = {
        "name": borrower.get("name", "Unknown"),
        **ratios,
        **scores,
        "total_score": total_score,
        "rating": rating,
        "risk_band": risk_band,
    }

    return result


# =========================================
# 2. Yahoo Finance helpers
# =========================================
def get_line_item(df: pd.DataFrame, labels) -> float:
    """
    Try multiple possible label names (since Yahoo can vary).
    Returns the latest column's value for the first label that matches.
    """
    if df is None or df.empty:
        return math.nan

    if isinstance(labels, str):
        labels = [labels]

    for label in labels:
        if label in df.index:
            latest_col = df.columns[0]  # most recent period
            value = df.loc[label, latest_col]
            try:
                return float(value)
            except (TypeError, ValueError):
                return math.nan

    return math.nan


def fetch_borrower_from_yahoo(ticker: str) -> Dict[str, Any]:
    t = yf.Ticker(ticker)

    fs = t.financials
    bs = t.balance_sheet
    cf = t.cashflow
    info = t.info

    # Pull key inputs
    ebitda = get_line_item(fs, ["Ebitda", "EBITDA"])
    ebit = get_line_item(fs, ["Ebit", "EBIT"])
    interest_expense = get_line_item(fs, ["Interest Expense", "InterestExpense", "Interest Expense Non Operating"])
    total_debt = get_line_item(bs, ["Total Debt", "TotalDebt"])
    current_assets = get_line_item(bs, ["Total Current Assets"])
    current_liabilities = get_line_item(bs, ["Total Current Liabilities"])

    # Debt repayment from cashflow (principal)
    debt_repayment = get_line_item(cf, ["Repayment Of Debt", "Debt Repayment"])

    # If we can't find repayment, assume simple 5% amortisation of total debt
    if math.isnan(debt_repayment) and not math.isnan(total_debt):
        debt_repayment = 0.05 * total_debt

    # Annual debt service = interest + principal
    if math.isnan(interest_expense) and math.isnan(debt_repayment):
        annual_debt_service = math.nan
    else:
        annual_debt_service = max(
            0.0,
            (0 if math.isnan(interest_expense) else interest_expense)
            + (0 if math.isnan(debt_repayment) else debt_repayment),
        )

    name = info.get("longName") or info.get("shortName") or ticker

    borrower = {
        "name": name,
        "ebitda": ebitda,
        "ebit": ebit,
        "interest_expense": interest_expense,
        "total_debt": total_debt,
        "current_assets": current_assets,
        "current_liabilities": current_liabilities,
        "annual_debt_service": annual_debt_service,
    }

    return borrower


# =========================================
# 3. Main: prompt for ticker and run scorecard
# =========================================
ticker = input("Please enter the ticker from Yahoo: ").strip().upper()

# Optional: inspect indices
t = yf.Ticker(ticker)
fs = t.financials
bs = t.balance_sheet
cf = t.cashflow
info = t.info

print("=== Income statement index ===")
print(fs.index)

print("\n=== Balance sheet index ===")
print(bs.index)

print("\n=== Cash flow index ===")
print(cf.index)

print("\nCompany name from info:")
print(info.get("longName") or info.get("shortName") or ticker)

# Now run the credit analysis
borrower = fetch_borrower_from_yahoo(ticker)
result = evaluate_borrower(borrower)

df = pd.DataFrame([result])

numeric_cols = [
    "dscr", "interest_coverage", "debt_ebitda", "current_ratio",
    "dscr_score", "interest_coverage_score", "debt_ebitda_score",
    "current_ratio_score", "total_score",
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].astype(float).round(2)

print("\n=== Credit Scorecard Result ===")
print(
    df[
        [
            "name",
            "dscr",
            "interest_coverage",
            "debt_ebitda",
            "current_ratio",
            "dscr_score",
            "interest_coverage_score",
            "debt_ebitda_score",
            "current_ratio_score",
            "total_score",
            "rating",
            "risk_band",
        ]
    ]
)
df.to_csv(f"credit_scorecard_{ticker}.csv", index=False)
print(f"\nCSV exported: credit_scorecard_{ticker}.csv")



Please enter the ticker from Yahoo: BHP.AX
=== Income statement index ===
Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA', 'Total Unusual Items',
       'Total Unusual Items Excluding Goodwill',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Interest Income',
       'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Total Operating Income As Reported',
       'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS',
       'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Discontinuous Operations',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
    